In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [3]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [4]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bbf41ec'}

In [5]:
from collections import defaultdict

In [6]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [7]:
len(hashes), len(documents)

(947, 948)

In [8]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

593f7569 2


In [9]:
hashes['593f7569']

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

In [10]:
import json

In [11]:
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [12]:
!head documents-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


In [13]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [14]:
from openai import OpenAI
client = OpenAI()

In [15]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [16]:
from tqdm.auto import tqdm

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
results = {}

In [18]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

100%|██████████| 948/948 [31:20<00:00,  1.98s/it]


In [19]:
import pickle

In [21]:
with open('results.bin', 'wb') as f_out:
    pickle.dump(results, f_out, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
# with open('results.bin', 'rb') as f_in:
#     results = pickle.load(f_in)

In [23]:
results['1f6520ca']

'[\n  "What background is needed to enroll in this course?",\n  "Are there any skills required before enrolling in this course?",\n  "Where can I find the prerequisites for this course?",\n  "What experience do I need before taking this course?",\n  "Can you tell me the necessary qualifications for this course?"\n]'

In [44]:
parsed_resulst = {}

for doc_id, json_questions in results.items():
    parsed_resulst[doc_id] = json.loads(json_questions)

In [37]:
doc_id

'e414df91'

In [39]:
print(results[doc_id].replace(r"\\", ""))

```json
[
    "How can I unzip a dataset in a Jupyter Notebook without seeing all the output messages?",
    "What command should I use to suppress unzipping output for a dataset folder in Jupyter Notebook?",
    "How do I unzip a zipped folder into a destination folder and suppress outputs within a Jupyter environment?",
    "What is the method for extracting a dataset from a zip file while avoiding verbose output through command line in Jupyter?",
    "How should I unzip and extract data in Jupyter Notebook without receiving a large volume of output messages?"
]
```


In [40]:
json_questions

'```json\n[\n    "How can I unzip a dataset in a Jupyter Notebook without seeing all the output messages?",\n    "What command should I use to suppress unzipping output for a dataset folder in Jupyter Notebook?",\n    "How do I unzip a zipped folder into a destination folder and suppress outputs within a Jupyter environment?",\n    "What is the method for extracting a dataset from a zip file while avoiding verbose output through command line in Jupyter?",\n    "How should I unzip and extract data in Jupyter Notebook without receiving a large volume of output messages?"\n]\n```'

In [41]:
json_questions = '[\n    "How can I unzip a dataset in a Jupyter Notebook without seeing all the output messages?",\n    "What command should I use to suppress unzipping output for a dataset folder in Jupyter Notebook?",\n    "How do I unzip a zipped folder into a destination folder and suppress outputs within a Jupyter environment?",\n    "What is the method for extracting a dataset from a zip file while avoiding verbose output through command line in Jupyter?",\n    "How should I unzip and extract data in Jupyter Notebook without receiving a large volume of output messages?"\n]'

In [42]:
json.loads(json_questions)

['How can I unzip a dataset in a Jupyter Notebook without seeing all the output messages?',
 'What command should I use to suppress unzipping output for a dataset folder in Jupyter Notebook?',
 'How do I unzip a zipped folder into a destination folder and suppress outputs within a Jupyter environment?',
 'What is the method for extracting a dataset from a zip file while avoiding verbose output through command line in Jupyter?',
 'How should I unzip and extract data in Jupyter Notebook without receiving a large volume of output messages?']

In [43]:
results[doc_id] = json_questions

In [45]:
doc_index = {d['id']: d for d in documents}

In [46]:
final_results = []

for doc_id, questions in parsed_resulst.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [47]:
import pandas as pd

In [48]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [49]:
df.to_csv('ground-truth-data.csv', index=False)

In [50]:
!head ground-truth-data.csv

question,course,document
When is the exact start date and time of the course?,data-engineering-zoomcamp,c02e79ef
How can I stay updated with course announcements?,data-engineering-zoomcamp,c02e79ef
Where do I register before the course begins?,data-engineering-zoomcamp,c02e79ef
Which platform hosts the first live session of the course?,data-engineering-zoomcamp,c02e79ef
How can I join the course community discussions?,data-engineering-zoomcamp,c02e79ef
What background is needed to enroll in this course?,data-engineering-zoomcamp,1f6520ca
Are there any skills required before enrolling in this course?,data-engineering-zoomcamp,1f6520ca
Where can I find the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
What experience do I need before taking this course?,data-engineering-zoomcamp,1f6520ca
